# Лабораторная работа 1. Прогнозирование моделью линейной регрессии

### 0. Загрузка данных

Перед нормированием и расчетом весов необходимо загрузить данные с помощью библиотеки numpy (```X``` - матрица признаков, ```y``` - матрица целевых значений).

In [166]:
import numpy as np

# получение данных из файлов
X = np.genfromtxt("./datasets/lab1_X.csv")
y = np.genfromtxt("./datasets/lab1_y.csv").reshape(50, 1)

print(f"Матрицы X и y:\n{X}\n{y}")


Матрицы X и y:
[[7.000e+00 1.590e+01 8.200e+00 5.100e+00 1.380e+01 2.290e+02 1.720e+02
  2.000e+01 2.104e+03]
 [7.200e+00 1.820e+01 7.400e+00 6.100e+00 1.430e+01 1.460e+02 1.670e+02
  2.910e+01 2.489e+03]
 [7.900e+00 1.970e+01 6.400e+00 4.700e+00 1.980e+01 1.740e+02 1.440e+02
  2.280e+01 2.428e+03]
 [7.700e+00 2.080e+01 6.900e+00 5.200e+00 1.710e+01 1.280e+02 1.110e+02
  4.270e+01 2.494e+03]
 [9.200e+00 1.590e+01 7.800e+00 5.300e+00 1.670e+01 1.690e+02 1.480e+02
  2.270e+01 2.094e+03]
 [7.600e+00 1.640e+01 6.700e+00 4.700e+00 1.550e+01 1.440e+02 1.500e+02
  2.790e+01 1.768e+03]
 [7.300e+00 1.830e+01 6.300e+00 4.900e+00 1.960e+01 1.380e+02 1.330e+02
  3.370e+01 1.982e+03]
 [7.900e+00 1.640e+01 6.800e+00 5.000e+00 1.760e+01 1.970e+02 1.550e+02
  2.660e+01 1.621e+03]
 [7.900e+00 1.700e+01 6.300e+00 4.400e+00 2.010e+01 1.820e+02 1.590e+02
  3.050e+01 1.631e+03]
 [8.000e+00 1.690e+01 8.200e+00 4.600e+00 1.550e+01 5.200e+02 1.970e+02
  1.910e+01 1.066e+03]
 [7.200e+00 1.760e+01 8.100e+00 5.2

Разделим все данные на выборку для обучения и выборку для валидации (```X_train```, ```y_train``` и ```X_test```, ```y_test``` соответственно)

In [167]:
# разделяем данные так: 70% для обучения, 30% для валидации
split = round(X.shape[0] * 0.7)

# Разделение для матрицы признаков
X_train = X[:split]
X_test = X[split:]

# Разделение для матрицы целевых значений
y_train = y[:split]
y_test = y[split:]

print(f"Размерности матриц: X_test - {X_test.shape}, X_train - {X_train.shape}, y_test - {y_test.shape}, y_train - {y_train.shape}")

Размерности матриц: X_test - (15, 9), X_train - (35, 9), y_test - (15, 1), y_train - (35, 1)


### 1. Нормирование (масштабирование) исходных данных.

Теперь необходимо нормализовать данные, поскольку разброс значенией данных слишком большой. Будет использоваться Z-нормализация

In [168]:
# функции для нахождения среднего и стандартного распределения
X_train = (X_train - np.mean(X_train)) / np.std(X_train)
X_test = (X_test - np.mean(X_test)) / np.std(X_test)

print(X_train)

[[-0.41047775 -0.39437582 -0.4083067  -0.41391524 -0.39817515 -0.00883409
  -0.11195881 -0.38695807  3.38342651]
 [-0.41011591 -0.39021464 -0.40975407 -0.41210603 -0.39727055 -0.15899816
  -0.12100484 -0.3704943   4.07997068]
 [-0.40884946 -0.38750084 -0.41156327 -0.41463892 -0.38731992 -0.1083404
  -0.16261657 -0.3818923   3.96960914]
 [-0.4092113  -0.38551071 -0.41065867 -0.41373432 -0.39220477 -0.19156386
  -0.22232036 -0.34588911  4.08901671]
 [-0.4064975  -0.39437582 -0.40903038 -0.4135534  -0.39292845 -0.11738643
  -0.15537975 -0.38207322  3.36533445]
 [-0.40939222 -0.39347121 -0.41102051 -0.41463892 -0.3950995  -0.16261657
  -0.15176134 -0.37266535  2.77553341]
 [-0.40993499 -0.39003372 -0.41174419 -0.41427708 -0.38768176 -0.17347181
  -0.18251784 -0.36217196  3.16270342]
 [-0.40884946 -0.39347121 -0.41083959 -0.41409616 -0.39130017 -0.06672867
  -0.14271531 -0.37501732  2.50958018]
 [-0.40884946 -0.39238569 -0.41174419 -0.41518168 -0.38677715 -0.09386676
  -0.13547849 -0.367961

### 2. Расчет весов линейной регрессии по аналитической формуле.

После нормализации можно рассчитать веса с помощью аналитической формулы (здесь также будут использоваться функции numpy)

In [169]:
cov_mat = np.matmul(np.transpose(X_train), X_train) # получение ковариационной матрицы
temp = np.matmul(np.linalg.inv(cov_mat), np.transpose(X_train)) # произведение обратной ковариационной матрицы и X транспонированной
w = np.matmul(temp, y_train) # получение матрицы весов

print(f"Полученные веса:\n{w}")

Полученные веса:
[[-105.34037223]
 [-458.9797658 ]
 [ 487.78769156]
 [-191.16376235]
 [ 110.10871375]
 [   2.60991367]
 [ -12.15662621]
 [  12.59801378]
 [   0.72561761]]


### 3. Построение и интепретация корреляционной матрицы. Определение степени мультиколлинеарности на основе числа обусловленности.

Теперь определим степень мультиколлинеарности на основе числа обусловленности. Будет использоваться функция ```eigvals``` из linalg для подсчета собственных значений.

In [170]:
print(f"Ковариационная матрица:\n{cov_mat}")
print(f"Число обусловленности: {np.linalg.cond(cov_mat)}")

Ковариационная матрица:
[[  5.77530015   5.61473232   5.83138838   5.90545595   5.56519485
    1.76908459   2.2683329    5.20386329 -38.04295817]
 [  5.61473232   5.46181614   5.67085926   5.74335203   5.41197705
    1.73367595   2.21566683   5.05750609 -36.80000384]
 [  5.83138838   5.67085926   5.8889558    5.96395566   5.62002904
    1.79516637   2.29662149   5.25364292 -38.33037116]
 [  5.90545595   5.74335203   5.96395566   6.04001011   5.69162012
    1.81849941   2.3269085    5.32016797 -38.77942472]
 [  5.56519485   5.41197705   5.62002904   5.69162012   5.36401479
    1.70867317   2.19131117   5.01451046 -36.55880406]
 [  1.76908459   1.73367595   1.79516637   1.81849941   1.70867317
    1.07744186   0.82449584   1.55892519 -11.97819387]
 [  2.2683329    2.21566683   2.29662149   2.3269085    2.19131117
    0.82449584   0.97668671   2.02953017 -14.61923909]
 [  5.20386329   5.05750609   5.25364292   5.32016797   5.01451046
    1.55892519   2.02953017   4.69693412 -34.31414729]


### 4. Анализ регрессионных остатков.

Найдем среднеквадратическую ошибку (MSE) для матрицы ```X_train```, будем искать регрессионные остатки относительно ```y_train```.

In [171]:
# MSE для обучающий данных
Eps_train = np.matmul(X_train, w) - y_train
MSE_train = np.matmul(np.transpose(Eps_train), Eps_train) / X.shape[0]

print(f"MSE: {MSE_train[0][0]}")

MSE: 1.7560210016827977


Теперь найдем MSE для валидационной выборки (```X_test``` и ```y_test```).

In [172]:
# MSE для валидационных данных
Eps_test = np.matmul(X_test, w) - y_test
MSE_test = np.matmul(np.transpose(Eps_test), Eps_test) / X.shape[0]

print(f"MSE: {MSE_test[0][0]}")

MSE: 12.078294172537214


### 5. Определение весов линейной регрессии градиентным методом. Проанализировать изменение ошибки от итерации к итерации

Сначала инициализурем весы (каждый параметр будет равен единице).

In [173]:
# 1. Инициализация весов
w = np.ones((X_train.shape[1], 1))

print(w)

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]


Теперь циклично будем расчитывать оценочные значения по весам, градиент функции потерь и новые значения весов. Эти шаги будут выполняться пока не будет слишком маленького изменения среднеквадратичной ошибки.

In [174]:
eps = 0.1
learning_rate = 0.1
S = []

for i in range(100):
    # 2. Расчет предсказанного значения y_pred по весам w
    y_pred = np.matmul(X_train, w)
    Eps_train = y_train - y_pred

    # 3. Расчет ошибки и градиента функции потерь
    S.append((1 / X_train.shape[0]) * np.matmul(np.transpose(Eps_train), Eps_train)[0][0])

    # 4. Установка новых значений весов
    dS = (-2 / X_train.shape[0]) * np.transpose(np.matmul(np.transpose(Eps_train), X_train))
    w -= learning_rate * dS

[print(f"Эпоха {_ + 1}: {S[_]}") for _ in range(len(S))]

Эпоха 1: 3521.4684261866596
Эпоха 2: 2263.06551740901
Эпоха 3: 1497.864551371973
Эпоха 4: 1030.754749796388
Эпоха 5: 743.8813684161681
Эпоха 6: 566.0527165048513
Эпоха 7: 454.2619379635492
Эпоха 8: 382.52612768092774
Эпоха 9: 335.1451301998233
Эпоха 10: 302.6298337716286
Эпоха 11: 279.2428441590897
Эпоха 12: 261.5130386807727
Эпоха 13: 247.33834578696852
Эпоха 14: 235.44381410815143
Эпоха 15: 225.05427722083775
Эпоха 16: 215.69663487127826
Эпоха 17: 207.08042303997954
Эпоха 18: 199.0256708750172
Эпоха 19: 191.41931924374956
Эпоха 20: 184.1888908119582
Эпоха 21: 177.28658032767598
Эпоха 22: 170.6796389703485
Эпоха 23: 164.34456056131918
Эпоха 24: 158.26356433148814
Эпоха 25: 152.42246503138998
Эпоха 26: 146.8093812081932
Эпоха 27: 141.41394993871785
Эпоха 28: 136.22684765682837
Эпоха 29: 131.2394960490145
Эпоха 30: 126.44387991099632
Эпоха 31: 121.8324328016301
Эпоха 32: 117.39796381253598
Эпоха 33: 113.13360933136819
Эпоха 34: 109.03280005484312
Эпоха 35: 105.08923736031896
Эпоха 36: 1

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

После использования градиентного спуска можно расчитать MSE для набора тестовых данных ```X_test``` и ```y_test```.

In [175]:
Eps_test = y_test - np.matmul(X_test, w) # регресионный остатки
MSE_test = (1 / X_test.shape[0]) * np.matmul(np.transpose(Eps_test), Eps_test) # среднеквадратичная ошибка

print(f"MSE: {MSE_test[0][0]}")

MSE: 18.08018066946566


### 6. Сравнение результатов по аналитическому и градиентному методу.

Если сравнивать ошибки на тестовых данных по аналитическому и градиентному методу, то можно увидеть, что результаты для первого метода лучше, чем для первого.

### 7. С помощью библиотеки sklearn сделать fit-predict модели линейной регрессии. Сравнить результаты с ранее полученными.

Теперь создадим модель через scikit-learn. Для этого воспользуемся одноименной библиотеки и функцией ```LinearRegression```.

In [180]:
from sklearn.linear_model import LinearRegression

# инициализация модели
model = LinearRegression()

# обучение модели
model.fit(X_train, y_train)

# получение оценочных значений
y_pred = model.predict(X_test)

# регрессионные остатки и MSE
Eps_test = y_test - y_pred
MSE_test = (1 / X_test.shape[0]) * np.matmul(np.transpose(Eps_test), Eps_test)

print(f"MSE: {MSE_test[0][0]}")

MSE: 27.512899496009197
